In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 14 17:05:30 2022

@author: haris
"""
import os


import numpy as np
# from Bio import SeqIO
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D,LayerNormalization, Dropout, Flatten, Dense, concatenate 
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import binary_accuracy
# from keras import regularizers
# import matplotlib.pyplot as plt
#from group_norm import GroupNormalization
from sklearn.metrics import confusion_matrix, recall_score, roc_curve, roc_auc_score, auc, average_precision_score
import math
import tensorflow as tf
from tensorflow.keras import losses
import random
# import scikitplot as skplt
# import seaborn as sn
import itertools
import pickle
np.random.seed(seed=21)
# ************ Data Processing ****************



import os
# from des_extraction import *
from feature_extraction import *

model_num=1                   # change
folder_num=14
# w_size=23


f_name=str(folder_num)

l= os.listdir('./')
skip=False
for list in l:
    if list =='results_'+f_name:
        skip=True
if skip==False:
    os.mkdir( 'results_'+f_name)

OutputDir = f'./results_{f_name}'


file = open(OutputDir + '/performance_ind.txt', 'w')


w_size=19
# for w_size in a1 :
list_1=l= os.listdir('./descriptor')
skip=False
for index_0 in list_1:
    if index_0 ==f'./descriptor/in_x_pssm_{w_size}.dat':
        skip=True
if skip==False:
    feature_extraction(w_size)




#




def listToString(s):  

    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1   




def prepareData():

    feature1 = pickle.load(open(f'./descriptor/train_x_pssm_{w_size}.dat',"rb"))
    feature2 = pickle.load(open(f'./descriptor/train_x_dssp_{w_size}.dat',"rb"))
    feature3 = pickle.load(open(f'./descriptor/train_x_hmm_{w_size}.dat',"rb"));
       

    label = pickle.load(open(f'./descriptor/train_y_{w_size}.dat',"rb"))

    
    return feature1,feature2,feature3, label



def funciton(PositiveCSV, NegativeCSV, OutputDir, folds):

    Positive_X, Positive_y, Negative_X, Negative_y = prepareData(PositiveCSV, NegativeCSV)





def calculateScore(X,XX,XXX, y, model, folds):
    
    score = model.evaluate([X,XX,XXX], y) # Gives loss and accuracy
    pred_y = model.predict([X,XX,XXX])

    accuracy = score[1];
    
    
    global acc1
    acc1=accuracy
    

    tempLabel = np.zeros(shape = y.shape, dtype=np.int32)

    for i in range(len(y)):
        if pred_y[i] < 0.5:
            tempLabel[i] = 0;
        else:
            tempLabel[i] = 1;
    # print(tempLabel)        
    
    confusion = confusion_matrix(y, tempLabel)
    TN, FP, FN, TP = confusion.ravel()

 
    sensitivity = TP / float(TP + FN)
    specificity = TN / float(TN + FP)
    from sklearn.metrics import matthews_corrcoef
    MCC=matthews_corrcoef(y, tempLabel)
    
    F1Score = (2 * TP) / float(2 * TP + FP + FN)
    precision = TP / float(TP + FP)

    pred_y = pred_y.reshape((-1, ))

    ROCArea = roc_auc_score(y, pred_y)
    AUPRC   = average_precision_score(y, pred_y)
    fpr, tpr, thresholds = roc_curve(y, pred_y)
    lossValue = None;

    print(y.shape)
    print(pred_y.shape)

    y_true = tf.convert_to_tensor(y, np.float32)
    y_pred = tf.convert_to_tensor(pred_y, np.float32)
    
    # plt.show() 
    
    lossValue = losses.binary_crossentropy(y_true, y_pred)#.eval()

    return {'sn' : sensitivity, 'sp' : specificity, 'acc' : accuracy, 'MCC' : MCC, 'AUC' : ROCArea,'AUPRC' : AUPRC, 'precision' : precision, 'F1' : F1Score}





os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

TF_ENABLE_ONEDNN_OPTS=0




#### indepentend dataset test_60.. descriptor  and labels###############
in_X_pssm=pickle.load(open(f'./descriptor/in_x_pssm_{w_size}.dat',"rb"))
in_X_dssp=pickle.load(open(f'./descriptor/in_x_dssp_{w_size}.dat',"rb"))
in_X_hmm=pickle.load(open(f'./descriptor/in_x_hmm_{w_size}.dat',"rb"))
in_y=pickle.load(open(f'./descriptor/in_y_{w_size}.dat',"rb")) # label







2024-02-27 14:07:02.900806: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-27 14:07:02.943579: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 14:07:02.943619: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 14:07:02.943659: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 14:07:02.953049: I tensorflow/core/platform/cpu_feature_g

In [3]:


def shuffleData(X, y, z):
    index = [i for i in range(len(X))]
    random.shuffle(index)
    X = X[index]
    y = y[index]
    z = z[index]
    return X, y ,z;


def shuffleData2(X1,X2,X3, y):
    index = [i for i in range(len(X1))]
    random.shuffle(index)
    X1 = X1[index]
    X2 = X2[index]
    X3 = X3[index]
    y = y[index]
    return X1, X2,X3, y;

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    
    return out


#### model no 1 change
def model_cnn(w_size):
    
    input_shape = (w_size,20) # One Hot
    inputs = Input(shape = input_shape)
    inputs2 = Input(shape = (w_size,14))  #NCPD
    inputs3 = Input(shape = (w_size,20))  #EIIP

    initializer = tf.keras.initializers.RandomUniform(seed=23)
    c1 = Conv1D(64,5, strides=1, activation='relu', input_shape=(w_size, 20))(inputs)
    c1 = Conv1D(32,3, strides=1, activation='relu')(c1)
    c1 = Conv1D(16,3, strides=1, activation='relu')(c1)

    c1 = LayerNormalization()(c1)
    # c1 = BatchNormalization()(c1)
    #c1 = MaxPooling1D(2,strides=3)(c1)
    c1 = Dropout(0.1,seed = 21)(c1)     # originally 0.1


    c2 = Conv1D(64,5, strides=1, activation='relu', input_shape=(w_size, 14))(inputs2)
    c2 = Conv1D(32,3, strides=1, activation='relu')(c2)
    c2 = Conv1D(16,3, strides=1, activation='relu')(c2)

    c2 = LayerNormalization()(c2)
    # c2 = BatchNormalization()(c2)
    #c2 = MaxPooling1D(2,strides=3)(c2)
    c2 = Dropout(0.3,seed = 21)(c2)    # originally 0.3

    c3 = Conv1D(64,5, strides=1, activation='relu', input_shape=(w_size, 20))(inputs3)
    c3 = Conv1D(32,3, strides=1, activation='relu')(c3)
    c3 = Conv1D(16,3, strides=1, activation='relu')(c3)

    c3 = LayerNormalization()(c3)
    
    # c3 = BatchNormalization()(c3)
    c3 = Dropout(0.2,seed = 21)(c3)   # originally 0.2

    # con0 = concatenate([c1,c2,c3])
    con0 = concatenate([c1,c2,c3])

    # cd = Dropout(0.20)(con0)
  

    # fc = Flatten()(cd)

    fc = Flatten()(con0)

    fc1 = Dense(256, activation='relu')(fc)
    #cd = Dropout(0.10)(fc1)

    fc1 = Dense(128, activation='relu')(fc1)

    fc1 = Dense(64, activation='relu')(fc1)
    
    fc2 = Dense(1, activation='sigmoid')(fc1)
    
    #model1 = Model(inputs =[ip], outputs = [fc2])
    
    #model1.compile(loss='kl_divergence', optimizer= 'adam', metrics=['accuracy'])
    model1 = Model(inputs =[inputs,inputs2,inputs3], outputs = [fc2])    
    opt=SGD(learning_rate=0.003, momentum = 0.99)

    model1.compile(loss='binary_crossentropy', optimizer= opt, metrics=['accuracy'])
    
    return model1





from sklearn.utils import compute_class_weight
train_1,train_2,train_3, train_label = prepareData()


i=0
j=0
k=0
for index in train_label:
    if index==1:
        i=i+1
    if index==0:
        j=j+1
    k=k+1      
    

positive_train_1 = np.zeros((i, w_size, 20))
positive_train_2 = np.zeros((i, w_size, 14))
positive_train_3 = np.zeros((i, w_size, 20))

negative_train_1= np.zeros((j, w_size, 20))
negative_train_2 = np.zeros((j, w_size, 14))
negative_train_3 = np.zeros((j, w_size, 20))

positive_train_label = np.zeros((i,))
negative_train_label = np.zeros((j,))








i=0
j=0
k=0
for index in train_label:
    if index==1:
        positive_train_label[i]=index
        positive_train_1[i]=train_1[k]
        positive_train_2[i]=train_2[k]
        positive_train_3[i]=train_3[k]        
        i=i+1
    if index==0:
        negative_train_label[j]=index
        negative_train_1[j]=train_1[k]
        negative_train_2[j]=train_2[k]
        negative_train_3[j]=train_3[k]
        
        j=j+1

    k=k+1


In [3]:


folds =5    
# random.shuffle(Positive_X);

# random.shuffle(Negitive_X);
positive_train_1,positive_train_2,positive_train_3=shuffleData(positive_train_1,positive_train_2,positive_train_3);
negative_train_1,negative_train_2,negative_train_3=shuffleData(negative_train_1,negative_train_2,negative_train_3);


Positive_X_1_Slices = chunkIt(positive_train_1, folds);
Positive_X_2_Slices = chunkIt(positive_train_2, folds);
Positive_X_3_Slices = chunkIt(positive_train_3, folds);
Positive_y_Slices = chunkIt(positive_train_label, folds);

Negative_X_1_Slices = chunkIt(negative_train_1, folds);
Negative_X_2_Slices = chunkIt(negative_train_2, folds);
Negative_X_3_Slices = chunkIt(negative_train_3, folds);
Negative_y_Slices = chunkIt(negative_train_label, folds);


testing_result = []

for test_index in range(folds):

    test_X_1 = np.concatenate((Positive_X_1_Slices[test_index],Negative_X_1_Slices[test_index]))
    test_X_2 = np.concatenate((Positive_X_2_Slices[test_index],Negative_X_2_Slices[test_index]))
    test_X_3 = np.concatenate((Positive_X_3_Slices[test_index],Negative_X_3_Slices[test_index]))    
    test_y = np.concatenate((Positive_y_Slices[test_index],Negative_y_Slices[test_index]))

    validation_index = (test_index+1) % folds;

    valid_X_1 = np.concatenate((Positive_X_1_Slices[validation_index],Negative_X_1_Slices[validation_index]))
    valid_X_2 = np.concatenate((Positive_X_2_Slices[validation_index],Negative_X_2_Slices[validation_index]))
    valid_X_3 = np.concatenate((Positive_X_3_Slices[validation_index],Negative_X_3_Slices[validation_index]))
    
    valid_y = np.concatenate((Positive_y_Slices[validation_index],Negative_y_Slices[validation_index]))
    
    
    
    start = 0;

    for val in range(0, folds):
        if val != test_index and val != validation_index:
            start = val;
            break;

    train_X_1 = np.concatenate((Positive_X_1_Slices[start],Negative_X_1_Slices[start]))
    train_X_2 = np.concatenate((Positive_X_2_Slices[start],Negative_X_2_Slices[start]))
    train_X_3 = np.concatenate((Positive_X_3_Slices[start],Negative_X_3_Slices[start]))
    train_y = np.concatenate((Positive_y_Slices[start],Negative_y_Slices[start]))

    for i in range(0, folds):
        if i != test_index and i != validation_index and i != start:
            tempX_1 = np.concatenate((Positive_X_1_Slices[i],Negative_X_1_Slices[i]))
            tempX_2 = np.concatenate((Positive_X_2_Slices[i],Negative_X_2_Slices[i]))
            tempX_3 = np.concatenate((Positive_X_3_Slices[i],Negative_X_3_Slices[i]))
            tempy = np.concatenate((Positive_y_Slices[i],Negative_y_Slices[i]))


            train_X_1 = np.concatenate((train_X_1, tempX_1))
            train_X_2 = np.concatenate((train_X_2, tempX_2))
            train_X_3 = np.concatenate((train_X_3, tempX_3))
            train_y = np.concatenate((train_y, tempy))
            

    test_X_1,test_X_2,test_X_3,test_y = shuffleData2(test_X_1,test_X_2,test_X_3,test_y);
    valid_X_1,valid_X_2,valid_X_3,valid_y = shuffleData2(valid_X_1,valid_X_2,valid_X_3,valid_y)
    train_X_1,train_X_2,train_X_3,train_y = shuffleData2(train_X_1,train_X_2,train_X_3,train_y);
    
 
    model = model_cnn(w_size);
    
    class_weights = compute_class_weight(class_weight = "balanced",classes = np.unique(train_y),y = train_y)
    
    class_weights = dict(enumerate(class_weights))      
    

    history = model.fit([train_X_1,train_X_2,train_X_3], train_y, batch_size = 64, class_weight=class_weights,epochs=80, validation_data = ([valid_X_1,valid_X_2,valid_X_3], valid_y));
    print(calculateScore(test_X_1,test_X_2,test_X_3, test_y, model, 1))
    model.save('my_model'+ str(test_index) +'.h5')


    

2024-02-26 16:55:14.133260: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 65692 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:1d:00.0, compute capability: 8.0


Epoch 1/80


2024-02-26 16:55:18.696012: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-02-26 16:55:19.388893: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-26 16:55:19.525986: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-26 16:55:21.315326: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f544c19e230 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-26 16:55:21.315366: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-02-26 16:55:21.423665: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


623/623 [==============================] - 19s 19ms/step - loss: 0.6943 - accuracy: 0.4948 - val_loss: 0.6868 - val_accuracy: 0.5998
Epoch 2/80
623/623 [==============================] - 11s 18ms/step - loss: 0.6752 - accuracy: 0.5368 - val_loss: 0.6630 - val_accuracy: 0.6213
Epoch 3/80
623/623 [==============================] - 11s 18ms/step - loss: 0.6629 - accuracy: 0.5564 - val_loss: 0.5987 - val_accuracy: 0.6641
Epoch 4/80
623/623 [==============================] - 11s 18ms/step - loss: 0.6492 - accuracy: 0.5613 - val_loss: 0.5160 - val_accuracy: 0.7438
Epoch 5/80
623/623 [==============================] - 11s 18ms/step - loss: 0.6419 - accuracy: 0.5675 - val_loss: 0.7380 - val_accuracy: 0.5654
Epoch 6/80
623/623 [==============================] - 11s 18ms/step - loss: 0.6386 - accuracy: 0.5571 - val_loss: 0.5431 - val_accuracy: 0.7159
Epoch 7/80
623/623 [==============================] - 11s 18ms/step - loss: 0.6370 - accuracy: 0.5440 - val_loss: 0.5983 - val_accuracy: 0.6546
Epo

/home/sharzil/anaconda3/envs/tf214/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

/home/sharzil/anaconda3/envs/tf214/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/80
623/623 [==============================] - 14s 18ms/step - loss: 0.6954 - accuracy: 0.5076 - val_loss: 0.7651 - val_accuracy: 0.2718
Epoch 2/80
623/623 [==============================] - 11s 18ms/step - loss: 0.6850 - accuracy: 0.4545 - val_loss: 0.6294 - val_accuracy: 0.8425
Epoch 3/80
623/623 [==============================] - 11s 18ms/step - loss: 0.6805 - accuracy: 0.4608 - val_loss: 0.6780 - val_accuracy: 0.5424
Epoch 4/80
623/623 [==============================] - 11s 18ms/step - loss: 0.6601 - accuracy: 0.5373 - val_loss: 0.6126 - val_accuracy: 0.5530
Epoch 5/80
623/623 [==============================] - 11s 18ms/step - loss: 0.6496 - accuracy: 0.5292 - val_loss: 0.7584 - val_accuracy: 0.4011
Epoch 6/80
623/623 [==============================] - 11s 18ms/step - loss: 0.6412 - accuracy: 0.5370 - val_loss: 0.6741 - val_accuracy: 0.4668
Epoch 7/80
623/623 [==============================] - 11s 18ms/step - loss: 0.6317 - accuracy: 0.5431 - val_loss: 0.7359 - val_accuracy:

/home/sharzil/anaconda3/envs/tf214/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/80
623/623 [==============================] - 14s 18ms/step - loss: 0.6956 - accuracy: 0.5151 - val_loss: 0.6480 - val_accuracy: 0.7261
Epoch 2/80
623/623 [==============================] - 11s 17ms/step - loss: 0.6775 - accuracy: 0.4913 - val_loss: 0.6185 - val_accuracy: 0.5424
Epoch 3/80
623/623 [==============================] - 11s 17ms/step - loss: 0.6688 - accuracy: 0.5128 - val_loss: 0.7582 - val_accuracy: 0.3142
Epoch 4/80
623/623 [==============================] - 11s 17ms/step - loss: 0.6614 - accuracy: 0.5229 - val_loss: 0.6051 - val_accuracy: 0.5820
Epoch 5/80
623/623 [==============================] - 11s 17ms/step - loss: 0.6556 - accuracy: 0.5021 - val_loss: 0.7587 - val_accuracy: 0.3579
Epoch 6/80
623/623 [==============================] - 11s 17ms/step - loss: 0.6448 - accuracy: 0.5448 - val_loss: 0.7377 - val_accuracy: 0.4118
Epoch 7/80
623/623 [==============================] - 11s 17ms/step - loss: 0.6372 - accuracy: 0.5299 - val_loss: 0.5511 - val_accuracy:

/home/sharzil/anaconda3/envs/tf214/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
test_1=pickle.load(open(f'./descriptor/in_x_pssm_{w_size}.dat',"rb"))
test_2=pickle.load(open(f'./descriptor/in_x_dssp_{w_size}.dat',"rb"))
test_3=pickle.load(open(f'./descriptor/in_x_hmm_{w_size}.dat',"rb"))
test_label=pickle.load(open(f'./descriptor/in_y_{w_size}.dat',"rb"))


In [6]:

new_model = tf.keras.models.load_model('model_19.h5')

print(calculateScore(test_1,test_2,test_3, test_label, new_model, 1))

2024-02-27 14:07:33.072049: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-02-27 14:07:33.730509: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-27 14:07:33.890206: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


411/411 [==============================] - 2s 5ms/step
(13144,)
(13144,)
{'sn': 0.6, 'sp': 0.8232902701237691, 'acc': 0.7880401611328125, 'MCC': 0.35958368044429523, 'AUC': 0.7842479648470112, 'AUPRC': 0.46693236111056585, 'precision': 0.3889409559512652, 'F1': 0.47194844579226686}
